## KNN from song_list_v7_binned.pkl

In [1]:
from joblib import dump, load
import numpy as np
import pickle
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, OrdinalEncoder

#### Read in data(frame)

In [2]:
song_df = pickle.load(open(r'C:\Users\mjhar\Downloads\song_list_v7_binned.pkl','rb'))

In [3]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555160 entries, 0 to 555159
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   songid            555160 non-null  object  
 1   availability      555160 non-null  float64 
 2   year              555160 non-null  int64   
 3   quin_bins         555160 non-null  category
 4   popularity        555160 non-null  object  
 5   acousticness      555160 non-null  float64 
 6   artist            555160 non-null  object  
 7   danceability      555160 non-null  float64 
 8   duration_ms       555160 non-null  float64 
 9   energy            555160 non-null  float64 
 10  instrumentalness  555160 non-null  float64 
 11  key               555160 non-null  float64 
 12  liveness          555160 non-null  float64 
 13  loudness          555160 non-null  float64 
 14  mode              555160 non-null  float64 
 15  speechiness       555160 non-null  float64 
 16  te

#### Replacing labels to b easily sorted for OrdinalEncode

In [4]:
replacements = {
    '90-95' : '1990-1995',
    '95-2000' : '1995-2000',
    '75-80' : '1975-1980',
    '85-90' : '1985-1990',
    '80-85' : '1980-1985',
    '50-60' : '1950-1960',
    '60-65' : '1960-1965',
    '>50s' : '0-1950',
    '65-70' : '1965-1970',
    '70-75' : '1970-1975',
    '2020' : '2020-2021'
}

for key, val in replacements.items():
    song_df['quin_bins'] = song_df['quin_bins'].replace(key,val)
    
cats = sorted(list(song_df['quin_bins'].unique()))